# Grafana Cloud Tutorial

Reference: grafana.com

mattcloudtech.grafana.net

## Linux Node Integration

### Prerequisites

In [ ]:
# Create a VM
!gcloud compute instances create monitoring --zone us-west1-a

In [ ]:
# SSH
# Open in another terminal
# gcloud compute ssh --zone "us-west1-a" "monitoring" --project "training1029"
# Check Distro: uname -a

In [ ]:
# ?

## Kubernetes Monitoring

### Prerequisites

In [ ]:
# Kubernetes CLI
!kubectl 

In [ ]:
%%sh
# Install Helm
# helm.sh/docs/intro/install
wget https://get.helm.sh/helm-v3.13.1-linux-amd64.tar.gz
tar -zxvf helm-v3.13.1-linux-amd64.tar.gz
sudo mv linux-amd64/helm /usr/local/bin/helm

In [ ]:
!helm

In [ ]:
%%sh
# Create a cluster
## Install Minkinube
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
# Start Minikube
minikube start

In [ ]:
!kubectl get all

In [ ]:
%%sh
helm repo add grafana https://grafana.github.io/helm-charts &&
  helm repo update &&
  helm upgrade --install grafana-k8s-monitoring grafana/k8s-monitoring \
    --namespace "default" --create-namespace --values - <<EOF
cluster:
  name: my-cluster
externalServices:
  prometheus:
    host: https://prometheus-prod-13-prod-us-east-0.grafana.net
    basicAuth:
      username: "1258617"
      password: glc_eyJvIjoiOTc2ODM3IiwibiI6Imt1YmVybmV0ZXMtdG9rZW4iLCJrIjoiOWhHMkw2NVRvU3QyenRLOUJuMzkzOE5aIiwibSI6eyJyIjoicHJvZC11cy1lYXN0LTAifX0=
  loki:
    host: https://logs-prod-006.grafana.net
    basicAuth:
      username: "728354"
      password: glc_eyJvIjoiOTc2ODM3IiwibiI6Imt1YmVybmV0ZXMtdG9rZW4iLCJrIjoiOWhHMkw2NVRvU3QyenRLOUJuMzkzOE5aIiwibSI6eyJyIjoicHJvZC11cy1lYXN0LTAifX0=
opencost:
  opencost:
    exporter:
      defaultClusterId: my-cluster
    prometheus:
      external:
        url: https://prometheus-prod-13-prod-us-east-0.grafana.net/api/prom
EOF

In [ ]:
!kubectl top 

In [ ]:
!kubectl get pods 

In [ ]:
!kubectl get deployments

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get all 

In [ ]:
!kubectl get namespaces

In [ ]:
### To test this let's build an app

In [ ]:
!pip install streamlit -q

In [ ]:
%%writefile app.py
import streamlit as st
st.text("A simple app.") 

In [ ]:
# On GCp, create a new firewall TCP:8501
!streamlit run app.py --server.address "0.0.0.0"

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
COPY . /app
RUN pip install streamlit -q
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
!docker build -t simple-app .

In [ ]:
!docker images

In [ ]:
!docker run -p 8501:8501 --name simple-app-container simple-app 

### (Option 1) Push to Artifact Registry 

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create docker-repo \
    --repository-format=docker \
    --location=us-west2 \
    --description="Docker repository"

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --tag us-west2-docker.pkg.dev/$(gcloud config get-value project)/docker-repo/simple-app:tag1

### (Option 2) Push to Docker Hub

Reference: hub.docker.com

In [ ]:
# docker login

In [ ]:
!docker build -t <DOCKER_HUB_ID>/simple_app .

In [ ]:
!docker push <DOCKER_HUB_ID>/simple_app

### Kubernetes Deployment and Service

In [ ]:
# Kubernetes

In [ ]:
!kubectl create namespace simple-app-namespace

In [ ]:
!kubectl get all --namespace simple-app

In [ ]:
!kubectl run simple-app --image=<DOCKER_HUB_ID>/simple_app

In [ ]:
!kubectl expose pod/simple-app --port 8501

In [ ]:
!kubectl get services 

In [ ]:
!kubectl port-forward service/simple-app 8501:8501 --address "0.0.0.0"